In [1]:
import ipywidgets
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("talk")

In [2]:
def deming_regression(x_anon, y, x_noise, y_noise):
    data = pd.DataFrame({'x': x_anon, 'y': y})
    x_mean = data['x'].mean()
    y_mean = data['y'].mean()
    cov = data.cov()
    s_xx = cov['x']['x']
    s_xy = cov['x']['y']
    s_yy = cov['y']['y']
    delta = (y_noise / x_noise) ** 2
    slope = (
        (s_yy - delta * s_xx + np.sqrt((s_yy - delta * s_xx) ** 2 + 4 * delta * s_xy ** 2)) /
        (2 * s_xy)
    )
    intercept = y_mean - slope * x_mean
    
    return intercept, slope

In [3]:
np.random.seed(42)

x = np.random.randint(0, 20, size=130)
y_noise = 2
gold_intercept = 5
gold_slope = 1
y = gold_intercept + gold_slope * x + np.random.normal(0, y_noise, size=x.shape)

@ipywidgets.interact(
    x_noise=ipywidgets.FloatSlider(value=2, min=0.2, max=4, step=0.05,
                                   layout=ipywidgets.Layout(width='700px'),
                                   continuous_update=False)
)
def show_deming(x_noise):
    np.random.seed(42)
    x_anon = x + np.random.normal(0, x_noise, size=x.shape)
    delta = (y_noise / x_noise) ** 2
    
    plt.figure(figsize=(10, 10))
    xlim = [-6, 20]
    ylim = [0, 26]
    plt.xlim(*xlim)
    plt.ylim(*ylim)
    sns.scatterplot(x=x,
                    y=y,
                    color='lightgray',
                    label='private data')
    plt.plot(xlim,
             gold_intercept + gold_slope * np.array(xlim),
             linestyle='--',
             c='lightgray',
             label='gold model')
    sns.scatterplot(x=x_anon,
                    y=y,
                    color='black',
                    zorder=9,
                    label='noised data')
    
    intercept, slope = deming_regression(x_anon, y, x_noise, y_noise) 
    sns.lineplot(x=xlim,
                 y=intercept + slope * np.array(xlim),
                 color='dodgerblue',
                 label='Deming')
    
    for idx, (x_datum, y_datum) in enumerate(zip(x_anon, y)):
        predicted_x = minimize(
            lambda x: (intercept + slope * x - y_datum) ** 2 + delta * (x_datum - x) ** 2,
            x0=3
        ).x[0]
        plt.plot([x_datum, predicted_x],
                 [y_datum, intercept + predicted_x * slope],
                 color='tomato',
                 label='error' if idx == 0 else None)
    plt.title(f'$\delta$ = {delta:.2f}')
    plt.legend()

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='x_noise', layout=Layout(wid…